In [1]:
import pandas as pd

import missions.W1.M3.etl_project_gdp as etl _basic
import missions.W1.M3.etl_project_gdp_with_sql as etl_sqlite

In [2]:
etl_basic.main()

[2025-01-09-10-59-23], [START] Starting GDP extraction process...
[2025-01-09-10-59-23], [START] Attempting to fetch the webpage: https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29
[2025-01-09-10-59-24], [COMPLETE] Successfully fetched the webpage: https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29
[2025-01-09-10-59-24], [START] Starting to parse the webpage: https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29 ...
[2025-01-09-10-59-24], [COMPLETE] Succesfuly finished parsing the webpage: https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29
[2025-01-09-10-59-24], [START] Saving data to missions/W1/M3/data/Countries_by_GDP.json ...
[2025-01-09-10-59-24], [COMPLETE] Data successfully saved to missions/W1/M3/data/Countries_by_GDP.json.
[2025-01-09-10-59-24], [COMPLETE] GDP extraction process completed successfully.


In [3]:
df_gdp = pd.read_json('missions/W1/M3/data/Countries_by_GDP.json')
df_gdp

,country,gdp,year,type
0,United States,"30,337,162",2025.0,IMF
1,China,"19,534,894",2025.0,IMF
2,Germany,"4,921,563",2025.0,IMF
3,Japan,"4,389,326",2025.0,IMF
4,India,"4,271,922",2025.0,IMF
...,...,...,...,...
204,Kiribati,311,2024.0,IMF
205,Palau,308,2024.0,IMF
206,Marshall Islands,305,2024.0,IMF
207,Nauru,161,2024.0,IMF


In [4]:
# GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.
df_gdp[df_gdp['gdp'] >= 100]

TypeError: '>=' not supported between instances of 'str' and 'int'

In [6]:
# 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.

df_region = pd.read_json('missions/W1/M3/data/cultural-geo-mapper.json')
merged = df_gdp.merge(df_region, on='country', how='left')

def top_n_mean(group, n=5):
    # 그룹 내 GDP 상위 n개 선택
    top_n = group.nlargest(n, 'gdp')
    return top_n['gdp'].mean()

merged.groupby('region').apply(top_n_mean, n=5).sort_values(ascending=False)

TypeError: Column 'gdp' has dtype object, cannot use method 'nlargest' with this dtype

In [69]:
import sqlite3
import pandas as pd
import etl_project_gdp_with_sql as etl_sql

def read_from_sqlite(db_name="data/Countries_by_GDP.db"):
    # SQLite DB 연결
    conn = sqlite3.connect(db_name)
    
    # SQL 쿼리로 테이블 읽기
    query = "SELECT * FROM Countries_by_GDP"
    list_of_gdp = pd.read_sql_query(query, conn)
    
    # 연결 닫기
    conn.close()
    
    return list_of_gdp

def analyze_gdp(list_of_gdp):
    # GDP_USD_billion이 100 이상인 데이터 필터링
    filtered_data = list_of_gdp[list_of_gdp['GDP_USD_billion'] >= 100]
    return filtered_data

list_of_gdp = read_from_sqlite()


,country,type,GDP_USD_billion,Year,Region
0,United States,IMF,30337.16,2025,North America
1,China,IMF,19534.89,2025,East Asia
2,Germany,IMF,4921.56,2025,Western Europe
3,Japan,IMF,4389.33,2025,East Asia
4,India,IMF,4271.92,2025,South Asia
...,...,...,...,...,...
204,Greenland,IMF,NaN,None,North America
205,Eritrea,IMF,NaN,None,Sub-Saharan Africa
206,Zanzibar,IMF,NaN,None,Sub-Saharan Africa
207,Sint Maarten,IMF,NaN,None,North America


In [60]:
with sqlite3.connect("data/Countries_by_GDP.db") as conn:
    cursor = conn.cursor()

    query = '''
        SELECT *
        FROM Countries_by_GDP
        WHERE GDP_USD_billion >= 100;
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    conn.rollback()

    cursor.close()

('United States', 'IMF', 30337.16, '2025', 'North America')
('China', 'IMF', 19534.89, '2025', 'East Asia')
('Germany', 'IMF', 4921.56, '2025', 'Western Europe')
('Japan', 'IMF', 4389.33, '2025', 'East Asia')
('India', 'IMF', 4271.92, '2025', 'South Asia')
('United Kingdom', 'IMF', 3730.26, '2025', 'Western Europe')
('France', 'IMF', 3283.43, '2025', 'Western Europe')
('Italy', 'IMF', 2459.6, '2025', 'Western Europe')
('Canada', 'IMF', 2330.31, '2025', 'North America')
('Brazil', 'IMF', 2307.16, '2025', 'Latin America')
('Russia', 'IMF', 2195.71, '2025', 'Eastern Europe')
('South Korea', 'IMF', 1947.13, '2025', 'East Asia')
('Australia', 'IMF', 1881.14, '2025', 'Oceania')
('Spain', 'IMF', 1827.58, '2025', 'Western Europe')
('Mexico', 'IMF', 1817.82, '2025', 'North America')
('Indonesia', 'IMF', 1492.62, '2025', 'Southeast Asia')
('Turkey', 'IMF', 1455.41, '2025', 'Eastern Europe')
('Netherlands', 'IMF', 1272.96, '2025', 'Western Europe')
('Saudi Arabia', 'IMF', 1136.58, '2025', 'Middle

In [61]:
with sqlite3.connect("data/Countries_by_GDP.db") as conn:
    cursor = conn.cursor()

    query = '''
        SELECT
        Region,
        ROUND(AVG(GDP_USD_billion), 2) AS Average_GDP_USD_billion
    FROM (
        SELECT
            Region,
            GDP_USD_billion,
            ROW_NUMBER() OVER (PARTITION BY Region ORDER BY GDP_USD_billion DESC) AS Rank
        FROM
            Countries_by_GDP
    )
    WHERE
        Rank <= 5
    GROUP BY
        Region
    ORDER BY
        Average_GDP_USD_billion DESC;
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    conn.rollback()

    cursor.close()

('North America', 8622.34)
('East Asia', 5421.57)
('Western Europe', 3244.49)
('Eastern Europe', 1066.6)
('South Asia', 1043.38)
('Latin America', 791.57)
('Southeast Asia', 711.89)
('Middle East', 598.13)
('Oceania', 436.66)
('Sub-Saharan Africa', 197.53)
('North Africa', 182.07)
('Central Asia', 106.39)


In [2]:
from bs4 import BeautifulSoup

html_content = """
<table>
    <tr class="static-row-header abc"><td>Header Row</td></tr>
    <tr><td>Row 1</td></tr>
    <tr><td>Row 2</td></tr>
    <tr class="static-row-header"><td>Another Header Row</td></tr>
    <tr><td>Row 3</td></tr>
</table>
"""

# HTML 파싱
soup = BeautifulSoup(html_content, 'html.parser')

# 특정 클래스 'static-row-header'를 포함하지 않는 <tr> 검색
table = soup.find("table")
rows = table.find_all("tr", class_=lambda x: x is None or "static-row-header" not in x)

# 결과 출력
for row in rows:
    print(row)


<tr class="static-row-header abc"><td>Header Row</td></tr>
<tr><td>Row 1</td></tr>
<tr><td>Row 2</td></tr>
<tr><td>Row 3</td></tr>
